<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.851 · Tipología y ciclo de vida de los datos · Práctica 1</p>
<p style="margin: 0; text-align:right;">2021-2 · Máster universitario en Ciencia de datos (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

******
   # Práctica 1: Web scraping
******

<div class="alert alert-block alert-info">
<strong>Autores: CARLOS MAS ESTÉVEZ Y CLAUDIA REINA FAJARDO</strong>
</div>

In [177]:
# Importamos las librerías necesarias
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

En primer lugar, haremos una prueba con la página principal de la web escogida, que nos ofrece la tarifa del presente día.

A través de esta toma de contacto, crearemos un dataset con tantas columnas como la fecha y el horario y con una sola fila con todos los precios del día.

In [178]:
# Fijamos url de la web
url = 'https://tarifaluzhora.es/'

# Hacemos la petición a la página
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [179]:
# Obtenemos las horas
horas = soup.find_all('span', itemprop="description")

# Obtenemos los precios
precios = soup.find_all('span', itemprop="price")

# Obtenemos la fecha
date = soup.find('input', {'name': 'date'}).get('value')

# Creamos un array con el contenido de las horas
columnas = ['fecha']
for h in horas:
    columnas.append(h.text)
    
# Creamos un array con el contenido de los precios
contenido = [date]
for p in precios:
    contenido.append(p.text)

In [180]:
# Creamos un dataset con los datos del primer día, cuyas columnas son la fecha y las horas
df = pd.DataFrame(data=[np.array(contenido)], columns=columnas)
df

,fecha,00h - 01h:,01h - 02h:,02h - 03h:,03h - 04h:,04h - 05h:,05h - 06h:,06h - 07h:,07h - 08h:,08h - 09h:,...,14h - 15h:,15h - 16h:,16h - 17h:,17h - 18h:,18h - 19h:,19h - 20h:,20h - 21h:,21h - 22h:,22h - 23h:,23h - 24h:
0,30/03/2022,0.33468 €/kWh,0.31902 €/kWh,0.30903 €/kWh,0.29593 €/kWh,0.29989 €/kWh,0.3092 €/kWh,0.32472 €/kWh,0.32824 €/kWh,0.37147 €/kWh,...,0.33943 €/kWh,0.33612 €/kWh,0.32761 €/kWh,0.31185 €/kWh,0.39006 €/kWh,0.4168 €/kWh,0.43635 €/kWh,0.44541 €/kWh,0.35615 €/kWh,0.35182 €/kWh


Cuando, mediante el formulario en forma de calendario de la web accedemos a una fecha anterior, se nos redirige a una url con el siguiente formato:
* https://tarifaluzhora.es/?tarifa=pcb&fecha={DD}%2F{MM}%2F{YYYY}
Donde:
* {DD} es el día con dos dígitos
* {MM} es el mes con dos dígitos
* {YYYY} es el año con cuatro dígitos
También hemos observado que no ofrece datos más allá del 01 de junio de 2021.

Con toda esta información procedemos a crear un array de urls que recorreremos para realizar la extracción y crear la base de datos.

In [181]:
# Creamos array vacio de urls
urls=[]

# Recorremos rango de fechas hacia atrás
for i in range(2022,2020,-1):
    # Si la fecha es 2022 solo queremos los tres primeros meses
    if i==2022:
        for j in range(3,0,-1):
            # Para febrero sólo recorremos 28 días
            if j==2:
                for k in range(28,0,-1):
                    url = 'https://tarifaluzhora.es/?tarifa=pcb&fecha='+str(k).zfill(2)+'%2F'+str(j).zfill(2)+'%2F'+str(i)
                    urls.append(url)
            else:
                # Para el resto de meses recorremos 31
                for k in range(31,0,-1):
                    url = 'https://tarifaluzhora.es/?tarifa=pcb&fecha='+str(k).zfill(2)+'%2F'+str(j).zfill(2)+'%2F'+str(i)
                    urls.append(url)
    # Si la fecha es 2021 solo queremos hasta junio
    else:
        for j in range(12,5,-1):
            # Para junio, septiembre y noviembre recorremos 30 días
            if (j==6) | (j==9) | (j==11):
                for k in range(30,0,-1):
                    url = 'https://tarifaluzhora.es/?tarifa=pcb&fecha='+str(k).zfill(2)+'%2F'+str(j).zfill(2)+'%2F'+str(i)
                    urls.append(url)
            else:
                # Para el resto recorremos 31 días
                for k in range(31,0,-1):
                    url = 'https://tarifaluzhora.es/?tarifa=pcb&fecha='+str(k).zfill(2)+'%2F'+str(j).zfill(2)+'%2F'+str(i)
                    urls.append(url)

In [182]:
# ELiminamos el 31 que es mañana y la fecha de hoy, que ya la tenemos
urls.pop(0)
urls.pop(0)

'https://tarifaluzhora.es/?tarifa=pcb&fecha=30%2F03%2F2022'

In [183]:
# Recorremos el array de urls
for i in urls:
    # Fijamos url de la web
    url = i

    # Hacemos la petición a la página
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Obtenemos los precios
    precios = soup.find_all('span', itemprop="price")

    # Obtenemos la fecha
    fecha = soup.find('input', {'name': 'date'}).get('value')
    
    # Creamos un array con el contenido de las horas
    columnas = ['fecha']
    for h in horas:
        columnas.append(h.text)

    # Creamos un array con el contenido de los precios
    contenido = [fecha]
    for p in precios:
        contenido.append(p.text)
        
    # Creamos el df
    df1 = pd.DataFrame(data=[np.array(contenido)], columns=columnas)
    
    # Lo unimos al df original para crear la bd
    df = pd.concat([df,df1])

In [184]:
# Cabecera del dataset
df.head()

,fecha,00h - 01h:,01h - 02h:,02h - 03h:,03h - 04h:,04h - 05h:,05h - 06h:,06h - 07h:,07h - 08h:,08h - 09h:,...,14h - 15h:,15h - 16h:,16h - 17h:,17h - 18h:,18h - 19h:,19h - 20h:,20h - 21h:,21h - 22h:,22h - 23h:,23h - 24h:
0,30/03/2022,0.33468 €/kWh,0.31902 €/kWh,0.30903 €/kWh,0.29593 €/kWh,0.29989 €/kWh,0.3092 €/kWh,0.32472 €/kWh,0.32824 €/kWh,0.37147 €/kWh,...,0.33943 €/kWh,0.33612 €/kWh,0.32761 €/kWh,0.31185 €/kWh,0.39006 €/kWh,0.4168 €/kWh,0.43635 €/kWh,0.44541 €/kWh,0.35615 €/kWh,0.35182 €/kWh
0,29/03/2022,0.30449 €/kWh,0.2824 €/kWh,0.27649 €/kWh,0.27374 €/kWh,0.27725 €/kWh,0.29108 €/kWh,0.32639 €/kWh,0.32426 €/kWh,0.36677 €/kWh,...,0.32994 €/kWh,0.32321 €/kWh,0.31876 €/kWh,0.33254 €/kWh,0.40217 €/kWh,0.40538 €/kWh,0.41921 €/kWh,0.42276 €/kWh,0.33883 €/kWh,0.32508 €/kWh
0,28/03/2022,0.28296 €/kWh,0.27426 €/kWh,0.26973 €/kWh,0.26371 €/kWh,0.26316 €/kWh,0.278 €/kWh,0.3068 €/kWh,0.30449 €/kWh,0.33477 €/kWh,...,0.2937 €/kWh,0.28725 €/kWh,0.2872 €/kWh,0.28162 €/kWh,0.39677 €/kWh,0.41407 €/kWh,0.45403 €/kWh,0.44684 €/kWh,0.34658 €/kWh,0.33382 €/kWh
0,27/03/2022,0.32519 €/kWh,0.30972 €/kWh,0 €/kWh,0.2992 €/kWh,0.28409 €/kWh,0.28504 €/kWh,0.2841 €/kWh,0.28608 €/kWh,0.28682 €/kWh,...,0.23267 €/kWh,0.22363 €/kWh,0.22486 €/kWh,0.2564 €/kWh,0.28313 €/kWh,0.32912 €/kWh,0.35556 €/kWh,0.34398 €/kWh,0.33106 €/kWh,0.30674 €/kWh
0,26/03/2022,0.33855 €/kWh,0.32707 €/kWh,0.32287 €/kWh,0.29904 €/kWh,0.28342 €/kWh,0.28992 €/kWh,0.30914 €/kWh,0.28685 €/kWh,0.2984 €/kWh,...,0.25066 €/kWh,0.25256 €/kWh,0.25645 €/kWh,0.29388 €/kWh,0.33031 €/kWh,0.36582 €/kWh,0.35917 €/kWh,0.3579 €/kWh,0.3411 €/kWh,0.33519 €/kWh


In [185]:
# Cola del dataset
df.tail()

,fecha,00h - 01h:,01h - 02h:,02h - 03h:,03h - 04h:,04h - 05h:,05h - 06h:,06h - 07h:,07h - 08h:,08h - 09h:,...,14h - 15h:,15h - 16h:,16h - 17h:,17h - 18h:,18h - 19h:,19h - 20h:,20h - 21h:,21h - 22h:,22h - 23h:,23h - 24h:
0,05/06/2021,0.1034 €/kWh,0.09697 €/kWh,0.09435 €/kWh,0.09548 €/kWh,0.09431 €/kWh,0.09451 €/kWh,0.09376 €/kWh,0.09609 €/kWh,0.10283 €/kWh,...,0.10482 €/kWh,0.10025 €/kWh,0.09899 €/kWh,0.09859 €/kWh,0.09919 €/kWh,0.09966 €/kWh,0.1073 €/kWh,0.11092 €/kWh,0.1113 €/kWh,0.10722 €/kWh
0,04/06/2021,0.10312 €/kWh,0.10295 €/kWh,0.10252 €/kWh,0.10581 €/kWh,0.10577 €/kWh,0.10825 €/kWh,0.11109 €/kWh,0.11252 €/kWh,0.14952 €/kWh,...,0.13715 €/kWh,0.13409 €/kWh,0.12952 €/kWh,0.13379 €/kWh,0.23188 €/kWh,0.23658 €/kWh,0.23832 €/kWh,0.23939 €/kWh,0.14442 €/kWh,0.14065 €/kWh
0,03/06/2021,0.10958 €/kWh,0.10608 €/kWh,0.10296 €/kWh,0.10445 €/kWh,0.10462 €/kWh,0.10753 €/kWh,0.11176 €/kWh,0.11282 €/kWh,0.14944 €/kWh,...,0.13265 €/kWh,0.12378 €/kWh,0.12341 €/kWh,0.12676 €/kWh,0.22634 €/kWh,0.23488 €/kWh,0.23745 €/kWh,0.24213 €/kWh,0.14956 €/kWh,0.14721 €/kWh
0,02/06/2021,0.1162 €/kWh,0.11437 €/kWh,0.11253 €/kWh,0.11287 €/kWh,0.11185 €/kWh,0.11197 €/kWh,0.11576 €/kWh,0.11564 €/kWh,0.15296 €/kWh,...,0.13851 €/kWh,0.13615 €/kWh,0.13227 €/kWh,0.13619 €/kWh,0.2312 €/kWh,0.23677 €/kWh,0.23927 €/kWh,0.245 €/kWh,0.15092 €/kWh,0.14831 €/kWh
0,01/06/2021,0.11633 €/kWh,0.11595 €/kWh,0.11489 €/kWh,0.11496 €/kWh,0.11484 €/kWh,0.11603 €/kWh,0.11629 €/kWh,0.1157 €/kWh,0.15289 €/kWh,...,0.13796 €/kWh,0.13288 €/kWh,0.13193 €/kWh,0.13599 €/kWh,0.23144 €/kWh,0.2404 €/kWh,0.2462 €/kWh,0.24808 €/kWh,0.15591 €/kWh,0.1565 €/kWh


A partir de aquí hay que limpiar los datos.

Podemos añadir columnas de media y desviación. 

Sacar mínimos, máximos y tendencias.

Hacer gráficas y predicciones.

...